In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install pyngrok
!pip install flask_ngrok
!pip3 install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 26.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=aef562dad6a14054efc4d2ca65f23ade436bb9eea0a37d4061577777bf5ad1d1
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 24 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=624f4a2f12a77dfa9e8fce03e41b3e53ca661c52013e7f6a9b9531f8f8ebb7f8
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Su

In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks/project local API"
!ls 


/content/drive/MyDrive/Colab Notebooks/project local API
'Flask API.ipynb'   helpers   README.md		 static
'Flask app .py'     model     requirements.txt	 video.mp4


#  Import libraries

In [4]:
from flask import Flask, render_template, request, redirect, url_for
from keras.models import load_model
import numpy as np
import cv2
import os
from flask import Flask
from werkzeug.utils import secure_filename
import os
from tensorflow import keras
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
import dlib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok


# Display Home Page 

In [5]:
app = Flask(__name__ , template_folder='static/templates')
ngrok.set_auth_token("29AhNyFJiBxEu0s07cR2zA8VqDQ_3wmwNStUSaHgpCiP2UNkd")
run_with_ngrok(app)
@app.route('/')
def main():
  return render_template ("Home.html")

#  Redirect To Upload 

In [6]:


@app.route('/upload_video', methods=['POST'])
def Video():
  return render_template ("Upload_Video.html")

@app.route('/upload_image', methods=['POST'])
def image():
    return render_template('Upload_image.html')
@app.route('/About-us', methods=['POST'])
def About_us():
  return render_template('About-us.html')


# Load Model 

In [7]:
import joblib
path = '/content/drive/MyDrive/Colab Notebooks/Experiments_DeepFakeDetection/finalized_model.sav'
model = joblib.load(path)
# model = tf.keras.models.load_model('/content/drive/MyDrive/Final project/Website/model/deepfake-detection-tensor.h5')

In [25]:
import sys
import os
import torch
import cv2
import time
from torch import nn
import matplotlib.pyplot as plt
import matplotlib
from torchvision.transforms import Normalize
gpu = torch.device('cuda:0'if torch.cuda.is_available() else 'cpu')
gpu

root = '/content/drive/MyDrive'
frames_per_video = 100
input_size = 224
test_val_frac = 0.3

sys.path.insert(0, os.path.join(root, 'Colab Notebooks', 'blazeface-pytorch'))
sys.path.insert(0, os.path.join(root, 'Colab Notebooks'))
from blazeface import BlazeFace
facedet = BlazeFace().to(gpu)

# Optionally change the thresholds:
facedet.min_score_thresh = 0.9
facedet.min_suppression_threshold = 0.9

facedet.load_weights(os.path.join(root, 'Colab Notebooks', "blazeface.pth"))
facedet.load_anchors(os.path.join(root, 'Colab Notebooks', "anchors.npy"))
_ = facedet.train(False)
from helpers.read_video_1 import VideoReader
from helpers.face_extract_1 import FaceExtractor

video_reader = VideoReader(verbose=True)
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [26]:
# from https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
import numpy as np

def azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    # Calculate the indices from the image
    y, x = np.indices(image.shape)

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

    r = np.hypot(x - center[0], y - center[1])

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]

    radial_prof = tbin / nr

    return radial_prof

# Function Prediction 

In [10]:
N = 500
from scipy.interpolate import griddata
def prediction_img (filepath):
  img = cv2.imread(filepath,0)
        
  # we crop the center
  h = int(img.shape[0]/3)
  w = int(img.shape[1]/3)
  img = img[h:-h,w:-w]

  f = np.fft.fft2(img)
  fshift = np.fft.fftshift(f)

  magnitude_spectrum = 20*np.log(np.abs(fshift))
  psd1D = azimuthalAverage(magnitude_spectrum)

  # Calculate the azimuthally averaged 1D power spectrum
  points = np.linspace(0,N,num=psd1D.size) # coordinates of a
  xi = np.linspace(0,N,num=N) # coordinates for interpolation

  interpolated = griddata(points,psd1D,xi,method='cubic')
  interpolated /= interpolated[0]
  pre = model.predict([interpolated])
  if(pre == 1):
    return 'Real'
  else:
    return 'Fake'

def prediction (filepath, rate =0.75):
  check = 0
  faces = face_extractor.process_video(filepath)
  face_extractor.keep_only_best_face(faces)
  if len(faces)>0:
      num=0
      for frame_data in faces:
          for img in frame_data['faces']:

            img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            h = int(img.shape[0]/3)
            w = int(img.shape[1]/3)
            img = img[h:-h,w:-w]

            f = np.fft.fft2(img)
            fshift = np.fft.fftshift(f)

            magnitude_spectrum = 20*np.log(np.abs(fshift))
            psd1D = azimuthalAverage(magnitude_spectrum)

            # Calculate the azimuthally averaged 1D power spectrum
            points = np.linspace(0,N,num=psd1D.size) # coordinates of a
            xi = np.linspace(0,N,num=N) # coordinates for interpolation

            interpolated = griddata(points,psd1D,xi,method='cubic')
            interpolated /= interpolated[0]
            if(int(model.predict([interpolated]))):
              check +=1
            else: 
              check -=1
  if(check <0):
    return('Fake')
  else:
    return("Real")

# Upload File

In [11]:
UPLOAD_FOLDER = 'static'
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

#  Video Prediction & Display Result 

In [12]:
@app.route('/predict_video', methods=['POST'])
def upload_video():
	file = request.files['file']
	filename = secure_filename(file.filename)
	file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
	filepath = "static/"+filename
	preds = prediction(filepath)
	return render_template("Display_Video.html",prediction =preds ,video_path = filename)

# Image Prediction & Display Result 

In [13]:
@app.route('/Predict_image', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        f = request.files['file']
        file_path = os.path.join (app.config['UPLOAD_FOLDER'], secure_filename(f.filename))
        f.save(file_path)
        # Make prediction
        preds = prediction_img(file_path)
    return render_template("Display_image.html",prediction = preds, img_path= f.filename )


# **Run Flask Application**#  

In [14]:
if __name__ =='__main__': 
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1b2e-34-145-225-206.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:27] "GET /static/css/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:27] "GET /static/css/normalize.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:28] "GET /static/background/deepfake2.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:28] "GET /static/background/cover.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:28] "GET /static/background/cover.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:29] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:29] "GET /static/background/cover.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:29] "POST /upload_video HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:30] "GET /static/js/jquery-1.12.4.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:48:30]

In [ ]:
from google.colab.patches import cv2_imshow
video_PATH = "/content/drive/MyDrive/Colab Notebooks/project local API/abc.mp4"
check = 0
faces = face_extractor.process_video(video_PATH)
face_extractor.keep_only_best_face(faces)
print(len(faces))
if len(faces)>0:
    for frame_data in faces:
      # scores = frame_data['scores'].cpu().numpy()
      scores = np.array(frame_data['scores'])
      print('Scores: ', scores)
      if(len(scores) > 0):
        img = frame_data['faces'][np.where(scores == np.amax(scores))[0][0]]
        # for img in frame_data['faces']:

        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        cv2_imshow(img)
        h = int(img.shape[0]/3)
        w = int(img.shape[1]/3)
        img = img[h:-h,w:-w]

        f = np.fft.fft2(img)
        fshift = np.fft.fftshift(f)

        magnitude_spectrum = 20*np.log(np.abs(fshift))
        psd1D = azimuthalAverage(magnitude_spectrum)

        # Calculate the azimuthally averaged 1D power spectrum
        points = np.linspace(0,N,num=psd1D.size) # coordinates of a
        xi = np.linspace(0,N,num=N) # coordinates for interpolation

        interpolated = griddata(points,psd1D,xi,method='cubic')
        interpolated /= interpolated[0]

        pre = model.predict([interpolated])
        print(pre)
        if(int(pre)):
          check +=1
        else: check -=1
print(check)
if(check < 0):
  print('FAKE')
else:
  print("REAL")